In [262]:
# Activate the julia environment in the working directory. 
using Pkg 
Pkg.activate(".")    # See Project.toml for packages loaded. 
Pkg.instantiate() 
Pkg.precompile() 
 
# Inculde pre-written utility files. 
include("ATUSDownload.jl")

  Activating project at `~/Repos/DoingRightNow-Analysis`


download_atus_data

# Examine the Respondent file

Information on ATUS files can be found [here](https://www.bls.gov/tus/data/datafiles-2021.htm). A PDF of the data dictionary can be found [here](https://www.bls.gov/tus/dictionaries/atusintcodebk21.pdf). A general user guide PDF can be found [here](www.bls.gov/tus/atususersguide.pdf). 
 
We'll start by looking at the Respondent file for
2021. Here's detail on the file from the ATUS website.
 
> About the Basic ATUS Data Files: 
> ATUS Respondent File 
> This file contains case-specific variables collected in ATUS (that is, variables for which there is one value for each respondent). These include, for example, labor force and earnings information, total time providing secondary childcare, and ATUS statistical weights. 
>  
> There is one record for each ATUS respondent. 
>  
> Below is a simplified example. The variable TUCASEID identifies each household, and the variable TULINENO identifies each individual within the household. The example contains responses from five individuals; note that the respondent always has TULINENO = 1. In the example, each respondent has a corresponding statistical weight for use in generating estimates representative of the U.S. civilian, noninstitutionalized population (TUFINLWGT), and values for school enrollment (TESCHENR), labor force status (TELFS), and total number of minutes spent alone on the diary day (TRTALONE). The actual ATUS Respondent file contains more variables and records. 
 
| TUCASEID       | TULINENO | TUFINLWGT   | TESCHENR | TELFS | TRTALONE | 
|----------------|----------|-------------|----------|-------|----------| 
| 20060101020210 | 1        | 22261358.19 | 1        | 1     | 40       | 
| 20060101020211 | 1        | 5019645.31  | 1        | 1     | 350      | 
| 20060101020212 | 1        | 2926068.74  | 1        | 5     | 0        | 
| 20060101020213 | 1        | 25780574.07 | 2        | 5     | 556      | 
| 20060101020214 | 1        | 3414645.94  | 1        | 4     | 100      | 


This is information about our population. So this is a good place to start. 
 
The 
 
There are some coded values we'll need to process. Specifically: 
-1
-2
-3

Weighting can be used for averages in the following manner. 
 
Let $\overline{T_j}$ be average time spend on activity $j$. This can be calculated as follows 
 
$$ 
\overline{T_j} = \frac{ \sum_i fwgt_i T_{ij} }{ \sum_i fwgt_i } 
$$ 
 
where $fwgt_i$ is teh 

Now let's pull the data and start looking at the population. 
 
I've already created a function 

In [263]:
resp2021 = download_atus_data("resp",2021) 
first(resp2021,10)

Row,TUCASEID,TULINENO,TUYEAR,TUMONTH,TEABSRSN,TEERN,TEERNH1O,TEERNH2,TEERNHRO,TEERNHRY,TEERNPER,TEERNRT,TEERNUOT,TEERNWKP,TEHRFTPT,TEHRUSL1,TEHRUSL2,TEHRUSLT,TEIO1COW,TEIO1ICD,TEIO1OCD,TELAYAVL,TELAYLK,TELFS,TELKAVL,TELKM1,TEMJOT,TERET1,TESCHENR,TESCHFT,TESCHLVL,TESPEMPNOT,TESPUHRS,TRCHILDNUM,TRDPFTPT,TRDTIND1,TRDTOCC1,TRERNHLY,TRERNUPD,TRERNWA,TRHERNAL,TRHHCHILD,TRHOLIDAY,TRIMIND1,TRMJIND1,TRMJOCC1,TRMJOCGR,TRNHHCHILD,TRNUMHOU,TROHHCHILD,TRSPFTPT,TRSPPRES,TRTALONE,TRTALONE_WK,TRTCC,TRTCCC,TRTCCC_WK,TRTCCTOT,TRTCHILD,TRTCOC,TRTEC,TRTFAMILY,TRTFRIEND,TRTHH,TRTHHFAMILY,TRTNOCHILD,TRTNOHH,TRTO,TRTOHH,TRTOHHCHILD,TRTONHH,TRTONHHCHILD,TRTSPONLY,TRTSPOUSE,TRTUNMPART,TRWERNAL,TRYHHCHILD,TTHR,TTOT,TTWK,TUABSOT,TUBUS,TUBUS1,TUBUS2OT,TUBUSL1,TUBUSL2,TUBUSL3,TUBUSL4,TUCC2,TUCC4,TUCC5B_CK,TUCC5_CK,TUCC9,TUDIARYDATE,TUDIARYDAY,TUDIS,TUDIS1,TUDIS2,TUECYTD,TUELDER,⋯
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String15,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,20210101210037,1,2021,1,-1,-1,1290,-1,40,1,1,-1,2,-1,-1,40,-1,40,4,4870,4720,-1,-1,1,-1,-1,2,-1,-1,-1,-1,-1,-1,0,1,22,16,1290,0,51600,0,2,0,7,5,4,3,2,1,2,-1,3,450,450,0,0,540,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,20210128,5,-1,-1,-1,-1,2,⋯
2,20210101210081,1,2021,1,-1,-1,1550,-1,37,1,1,-1,2,-1,-1,34,-1,34,4,5480,4700,-1,-1,1,-1,-1,2,-1,2,-1,-1,1,27,1,2,22,16,1550,0,57350,0,1,0,7,5,4,3,2,3,1,2,2,330,330,600,0,0,600,540,0,-1,540,0,600,540,0,0,600,600,540,0,0,0,0,480,0,10,0,0,0,-1,2,-1,-1,-1,-1,-1,-1,-2,19:30:00,-1,-1,-1,20210130,7,-1,-1,-1,-1,1,⋯
3,20210101210099,1,2021,1,-1,-1,-1,-1,-1,2,6,2,1,26,-1,40,-1,40,4,770,6260,-1,-1,1,-1,-1,2,-1,2,-1,-1,1,40,0,1,4,19,-1,0,173076,-1,2,0,3,3,7,5,2,2,2,1,1,130,130,0,15,520,0,0,0,-1,230,0,0,230,0,0,0,0,0,0,0,230,230,0,0,-1,0,0,0,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,20210128,5,-1,-1,-1,-1,2,⋯
4,20210101210101,1,2021,1,-1,-1,-1,-1,-1,2,2,2,1,-1,-1,40,3,43,4,6380,5510,-1,-1,1,-1,-1,1,-1,2,-1,-1,-1,-1,0,1,23,17,-1,1,60000,-1,2,0,8,6,5,3,2,1,2,-1,3,445,445,0,0,0,0,0,0,-1,0,220,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,20210129,6,-1,-1,-1,-1,2,⋯
5,20210101210109,1,2021,1,-1,-1,-1,1250,-1,1,2,1,2,-1,-1,35,-1,35,4,6380,9640,-1,-1,1,-1,-1,2,-1,2,-1,-1,-1,-1,0,1,23,22,1250,1,28000,0,2,0,8,6,10,6,2,2,2,-1,3,425,425,0,0,0,0,100,0,-1,160,0,0,160,100,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,20210130,7,-1,-1,-1,-1,2,⋯
6,20210101210115,1,2021,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,5,-1,-1,-1,-1,1,1,1,-1,-1,4,-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,-1,2,5,2,-1,3,525,525,0,0,0,0,375,0,-1,375,0,0,375,375,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-2,-2,1,-1,-1,20210124,1,-1,-1,-1,-1,2,⋯
7,20210101210131,1,2021,1,-1,-1,1500,-1,40,1,1,-1,2,-1,-1,40,-1,40,4,4690,9610,-1,-1,1,-1,-1,2,-1,2,-1,-1,-1,-1,0,1,22,22,1500,0,60000,0,2,0,7,5,10,6,2,1,2,-1,3,660,660,0,0,0,0,0,0,-1,215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,20210124,1,-1,-1,-1,-1,2,⋯
8,20210101210139,1,2021,1,11,-1,-1,1600,-1,1,6,1,2,10,2,-4,-1,-4,4,3570,8650,-1,-1,2,-1,-1,2,-1,1,1,1,-1,-1,3,2,10,21,1600,1,20000,0,1,0,4,4,9,6,2,7,2,-1,3,180,180,0,0,0,0,450,0,-1,450,0,0,450,450,0,0,0,0,0,0,0,0,0,0,12,0,0,0,1,2,-1,-1,-1,-1,-1,-1,09:00:00,20:30:00,1,-1,-1,20210124,1,-1,-1,-1,-1,2,⋯
9,20210101210155,1,2021,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,1,1,-1,-1,1,1,1,-1,-1,3,-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,-1,2,5,2,-1,3,325,325,0,0,0,0,

In [116]:
# Replace -1, -2, and -3 with missing. 
allowmissing!(resp2021,names(resp2021)[2:end]) 
map(eachcol(resp2021)) do col 
    replace!(col,-1=>missing,-2=>missing,-3=>missing) 
end

175-element Vector{AbstractVector}:
 [20210101210037, 20210101210081, 20210101210099, 20210101210101, 20210101210109, 20210101210115, 20210101210131, 20210101210139, 20210101210155, 20210101210175  …  20211212212101, 20211212212120, 20211212212183, 20211212212328, 20211212212471, 20211212212479, 20211212212502, 20211212212555, 20211212212567, 20211212212591]
 Union{Missing, Int64}[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Union{Missing, Int64}[2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021  …  2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021]
 Union{Missing, Int64}[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
 Union{Missing, Int64}[missing, missing, missing, missing, missing, missing, missing, 11, missing, missing  …  missing, missing, missing, missing, missing, 4, missing, missing, missing, missing]
 Union{Missing, Int64}[missing, missing, missing, missing, missing, missing, missing, missing, missing, missing  …  

In [264]:
size(resp2021)

(9087, 175)

This is a big table. And the codes are a little tricky to parse. It's best to go through the data dictionary and find the interesting ones.

## Earnings 
 


In [265]:
smry_pct_with_teern = sum(ismissing.(resp2021.TEERN))/size(resp2021)[1] 
"Proportion of records with missing data for TEERN: $(round(smry_pct_with_teern*100, digits = 1))%"

"Proportion of records with missing data for TEERN: 0.0%"

Unfortunately, we're missing a lot of data for that field. Let's see which fields have decent non-missing coverage. Specifically, let's focus on columns with >90% coverage.

In [268]:
column_missing_coverage = transform(describe(resp2021,:nmissing), :nmissing => (x -> 1 .- x./nrow(resp2021)) => :pct_not_missing) 
non_missing_cols = column_missing_coverage[column_missing_coverage.pct_not_missing .>= 0.90, :variable] 
size(non_missing_cols)

(175,)

There are
49 fields with full coverage.

In [269]:
print(non_missing_cols)

[:TUCASEID, :TULINENO, :TUYEAR, :TUMONTH, :TEABSRSN, :TEERN, :TEERNH1O, :TEERNH2, :TEERNHRO, :TEERNHRY, :TEERNPER, :TEERNRT, :TEERNUOT, :TEERNWKP, :TEHRFTPT, :TEHRUSL1, :TEHRUSL2, :TEHRUSLT, :TEIO1COW, :TEIO1ICD, :TEIO1OCD, :TELAYAVL, :TELAYLK, :TELFS, :TELKAVL, :TELKM1, :TEMJOT, :TERET1, :TESCHENR, :TESCHFT, :TESCHLVL, :TESPEMPNOT, :TESPUHRS, :TRCHILDNUM, :TRDPFTPT, :TRDTIND1, :TRDTOCC1, :TRERNHLY, :TRERNUPD, :TRERNWA, :TRHERNAL, :TRHHCHILD, :TRHOLIDAY, :TRIMIND1, :TRMJIND1, :TRMJOCC1, :TRMJOCGR, :TRNHHCHILD, :TRNUMHOU, :TROHHCHILD, :TRSPFTPT, :TRSPPRES, :TRTALONE, :TRTALONE_WK, :TRTCC, :TRTCCC, :TRTCCC_WK, :TRTCCTOT, :TRTCHILD, :TRTCOC, :TRTEC, :TRTFAMILY, :TRTFRIEND, :TRTHH, :TRTHHFAMILY, :TRTNOCHILD, :TRTNOHH, :TRTO, :TRTOHH, :TRTOHHCHILD, :TRTONHH, :TRTONHHCHILD, :TRTSPONLY, :TRTSPOUSE, :TRTUNMPART, :TRWERNAL, :TRYHHCHILD, :TTHR, :TTOT, :TTWK, :TUABSOT, :TUBUS, :TUBUS1, :TUBUS2OT, :TUBUSL1, :TUBUSL2, :TUBUSL3, :TUBUSL4, :TUCC2, :TUCC4, :TUCC5B_CK, :TUCC5_CK, :TUCC9, :TUDIARYDATE, 

So which ones are interesting? 
TELFS
TRCHILDNUM
TRHHCHILD
TRNUMHOU
TRSPPRES
TRTALONE
TRTALONE_WK
TTHR
TTWK
TUFWK

This data isn't what I actually thought it was. It's not information about the respondents. It's the summary of how they spent their time. (Similar to the Activity summary file, but less granular.)
 
What I actually want is the roster file.

## Roster File

In [270]:
rost2021 = download_atus_data("rost",2021)

Row,TUCASEID,TULINENO,TEAGE,TERRP,TESEX,TXAGE,TXRRP,TXSEX
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,20210101210037,1,60,19,2,0,0,0
2,20210101210081,1,30,18,2,0,0,0
3,20210101210081,2,29,21,1,0,0,0
4,20210101210081,3,10,22,1,0,0,0
5,20210101210099,1,43,18,1,0,0,0
6,20210101210099,2,39,20,2,0,0,0
7,20210101210101,1,23,19,1,0,0,0
8,20210101210109,1,20,19,1,0,0,0
9,20210101210109,2,40,24,2,0,0,0


No, that's not what I want either. What about the CPS file? It has its own data dictionary [here](https://www.bls.gov/tus/dictionaries/atuscpscodebk21.pdf).

In [271]:
cps2021 = download_atus_data("cps",2021); 
describe(cps2021)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,TUCASEID,2.02107e13,20210101210030,2.02107e13,20211212212591,0,Int64
2,TULINENO,2.25069,1,2.0,14,0,Int64
3,GEREG,2.71391,1,3.0,4,0,Int64
4,GEDIV,5.32372,1,5.0,9,0,Int64
5,GESTFIPS,27.6538,1,27.0,56,0,Int64
6,GTMETSTA,1.15927,1,1.0,3,0,Int64
7,GTCBSA,24650.6,0,27260.0,49740,0,Int64
8,GTCO,32.8323,0,0.0,810,0,Int64
9,HEFAMINC,11.9373,1,13.0,16,0,Int64


This is what I was looking for!!! This has a BUNCH of family demographic information, like income, region where they live, type of housing, etc.
 
Let's see what kind of coverage we have.

In [272]:
# Replace -1, -2, and -3 with missing. 
allowmissing!(cps2021,names(cps2021)[3:end]) 
map(eachcol(cps2021)) do col 
    replace!(col,-1=>missing,-2=>missing,-3=>missing) 
end;

In [274]:
cps2021_coverage = transform(describe(cps2021,:nmissing),:nmissing => (x -> 1 .- x ./ nrow(cps2021)) => :pct_not_missing)

Row,variable,nmissing,pct_not_missing
,Symbol,Int64,Float64
1,TUCASEID,0,1.0
2,TULINENO,0,1.0
3,GEREG,0,1.0
4,GEDIV,0,1.0
5,GESTFIPS,0,1.0
6,GTMETSTA,0,1.0
7,GTCBSA,0,1.0
8,GTCO,0,1.0
9,HEFAMINC,0,1.0


Looks like there are like
100 or so fields with over
75% coverage.

Here's two important fields:  
TRATUSR 
TULINENO 
 
And here are the other interesting fields: 
GEREG 
GEDIV 
GESTFIPS 
GTCBSA 
GTCO 
GTMETSTA 
HEFAMINC 
HEHOUSUT 
HETENURE 
HRHTYPE 
PECYC 
PEEDUCA 
PEDWWK 
PEDW4WK 
PRTAGE 
PRWKSTAT 
PUABSOT 
 


We won't look at all of these, but let'st take a look at some.

In [276]:
cps_want_cols = ["TRATUSR", "TULINENO", "GEREG", "GEDIV", "GESTFIPS", "GTCBSA", "GTCO", "GTMETSTA", "HEFAMINC", "HEHOUSUT", "HETENURE", "HRHTYPE", "PECYC", "PEEDUCA", "PEDWWK", "PEDW4WK", "PRTAGE", "PRWKSTAT", "PUABSOT"]

19-element Vector{String}:
 "TRATUSR"
 "TULINENO"
 "GEREG"
 "GEDIV"
 "GESTFIPS"
 "GTCBSA"
 "GTCO"
 "GTMETSTA"
 "HEFAMINC"
 "HEHOUSUT"
 "HETENURE"
 "HRHTYPE"
 "PECYC"
 "PEEDUCA"
 "PEDWWK"
 "PEDW4WK"
 "PRTAGE"
 "PRWKSTAT"
 "PUABSOT"

In [279]:
cps = cps2021[:,cps_want_cols]

Row,TRATUSR,TULINENO,GEREG,GEDIV,GESTFIPS,GTCBSA,GTCO,GTMETSTA,HEFAMINC,HEHOUSUT,HETENURE,HRHTYPE,PECYC,PEEDUCA,PEDWWK,PEDW4WK,PRTAGE,PRWKSTAT,PUABSOT
,Int64?,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,0,1,2,4,27,0,0,2,1,1,2,7,missing,39,missing,missing,80,1,missing
2,0,1,3,7,5,38220,0,1,7,5,1,1,missing,39,missing,missing,80,1,missing
3,0,2,3,7,5,38220,0,1,7,5,1,1,3,41,missing,missing,75,1,missing
4,1,1,4,8,8,0,0,2,15,5,1,7,missing,39,missing,missing,60,2,missing
5,1,1,2,3,17,40420,0,1,10,1,1,6,5,42,missing,missing,30,2,missing
6,0,2,2,3,17,40420,0,1,10,1,1,6,1,40,missing,missing,29,7,missing
7,0,3,2,3,17,40420,0,1,10,1,1,6,missing,missing,missing,missing,10,missing,missing
8,0,1,3,7,48,41700,0,1,6,1,2,7,missing,39,missing,missing,63,1,missing
9,1,1,4,8,4,38060,13,1,15,1,1,1,missing,39,missing,missing,43,2,missing


In [280]:
using CategoricalArrays

In [282]:
cps.GEREG = categorical(cps.GEREG)

63645-element CategoricalArray{Union{Missing, Int64},1,UInt32}:
 2
 3
 3
 4
 2
 2
 2
 3
 4
 4
 ⋮
 2
 1
 1
 1
 1
 3
 3
 3
 3

In [283]:
cps

Row,TRATUSR,TULINENO,GEREG,GEDIV,GESTFIPS,GTCBSA,GTCO,GTMETSTA,HEFAMINC,HEHOUSUT,HETENURE,HRHTYPE,PECYC,PEEDUCA,PEDWWK,PEDW4WK,PRTAGE,PRWKSTAT,PUABSOT
,Int64?,Int64,Cat…?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,0,1,2,4,27,0,0,2,1,1,2,7,missing,39,missing,missing,80,1,missing
2,0,1,3,7,5,38220,0,1,7,5,1,1,missing,39,missing,missing,80,1,missing
3,0,2,3,7,5,38220,0,1,7,5,1,1,3,41,missing,missing,75,1,missing
4,1,1,4,8,8,0,0,2,15,5,1,7,missing,39,missing,missing,60,2,missing
5,1,1,2,3,17,40420,0,1,10,1,1,6,5,42,missing,missing,30,2,missing
6,0,2,2,3,17,40420,0,1,10,1,1,6,1,40,missing,missing,29,7,missing
7,0,3,2,3,17,40420,0,1,10,1,1,6,missing,missing,missing,missing,10,missing,missing
8,0,1,3,7,48,41700,0,1,6,1,2,7,missing,39,missing,missing,63,1,missing
9,1,1,4,8,4,38060,13,1,15,1,1,1,missing,39,missing,missing,43,2,missing
